# Spectator/TLS example

In [1]:
from c3.experiment import Experiment

## Optimal control simulation

In [5]:
exp = Experiment()
exp.read_config("two_qubits.c3exp")
parameter_map = exp.pmap

In [6]:
import numpy as np
import copy
def generate_gateset(single_q_gate):
    gates = [single_q_gate]
    angles = [np.pi/2, np.pi, 3*np.pi/2]
    names = ["ry90p", "rx90m", "ry90m"]
    for name, angle in zip(names, angles):
        gate = copy.deepcopy(single_q_gate)
        gate.set_name(name)
        for chan in gate.comps:
            for env in gate.comps[chan].values():
                if "xy_angle" in env.params:
                    env.params["xy_angle"].set_value(angle, extend_bounds=True)
        gates.append(gate)
    return gates

In [7]:
gates = []
rx90p_q1 = parameter_map.instructions["rx90p[0]"]
gates.extend(generate_gateset(rx90p_q1))
rx90p_q1 = parameter_map.instructions["rx90p[0]"]
gates.extend(generate_gateset(rx90p_q1))


In [8]:
for gate in gates:
    parameter_map.instructions[gate.get_key()] = gate
parameter_map.update_parameters()

In [9]:
parameter_map.get_full_params()

{'Q1-Q2-strength': 50.000 MHz 2pi,
 'Q1-freq': 5.000 GHz 2pi,
 'Q1-anhar': -210.000 MHz 2pi,
 'Q1-t1': 27.000 µs,
 'Q1-t2star': 39.000 µs,
 'Q1-temp': 50.000 mK,
 'Q2-freq': 5.600 GHz 2pi,
 'Q2-anhar': -240.000 MHz 2pi,
 'Q2-t1': 23.000 µs,
 'Q2-t2star': 31.000 µs,
 'Q2-temp': 50.000 mK,
 'v_to_hz-V_to_Hz': 1.000 GHz/V,
 'cx[0, 1]-d1-gauss1-amp': 800.000 mV,
 'cx[0, 1]-d1-gauss1-delta': -1.000 ,
 'cx[0, 1]-d1-gauss1-freq_offset': -53.000 MHz 2pi,
 'cx[0, 1]-d1-gauss1-xy_angle': -444.089 arad,
 'cx[0, 1]-d1-gauss1-sigma': 11.250 ns,
 'cx[0, 1]-d1-gauss1-t_final': 45.000 ns,
 'cx[0, 1]-d1-carrier-freq': 5.650 GHz 2pi,
 'cx[0, 1]-d1-carrier-framechange': 4.712 rad,
 'cx[0, 1]-d2-gauss2-amp': 30.000 mV,
 'cx[0, 1]-d2-gauss2-delta': -1.000 ,
 'cx[0, 1]-d2-gauss2-freq_offset': -53.000 MHz 2pi,
 'cx[0, 1]-d2-gauss2-xy_angle': -444.089 arad,
 'cx[0, 1]-d2-gauss2-sigma': 11.250 ns,
 'cx[0, 1]-d2-gauss2-t_final': 45.000 ns,
 'cx[0, 1]-d2-carrier-freq': 5.650 GHz 2pi,
 'cx[0, 1]-d2-carrier-framec

In [7]:
import os
import tempfile
from c3.optimizers.optimalcontrol import OptimalControl
from c3.libraries import algorithms, fidelities

log_dir = os.path.join(tempfile.TemporaryDirectory().name, "c3logs")
opt = OptimalControl(
    dir_path=log_dir,
    fid_func=fidelities.unitary_infid_set,
    fid_subspace=["Q1", "Q2"],
    pmap=parameter_map,
    algorithm=algorithms.lbfgs,
    options={
        "maxfun": 25
    },
    run_name="cnot12"
)
opt_gates = ["rx90p[0]", "rx90p[1]"]
exp.set_opt_gates(opt_gates)

two_qubit_gateset_opt_map=[
    [("cx[0, 1]", "d1", "gauss1", "amp")],
    [("cx[0, 1]", "d1", "gauss1", "freq_offset")],
    [("cx[0, 1]", "d1", "gauss1", "xy_angle")],
    [("cx[0, 1]", "d1", "gauss1", "delta")],
    [("cx[0, 1]", "d1", "carrier", "framechange")],
    [("cx[0, 1]", "d2", "gauss2", "amp")],
    [("cx[0, 1]", "d2", "gauss2", "freq_offset")],
    [("cx[0, 1]", "d2", "gauss2", "xy_angle")],
    [("cx[0, 1]", "d2", "gauss2", "delta")],
    [("cx[0, 1]", "d2", "carrier", "framechange")],
]

gateset_opt_map=[
    [("rx90p[0]", "d1", "gauss", "amp")],
    [("rx90p[0]", "d1", "gauss", "freq_offset")],
    [("rx90p[0]", "d1", "gauss", "xy_angle")],
    [("rx90p[0]", "d1", "gauss", "delta")],
    [("rx90p[0]", "d1", "carrier", "framechange")],
    [("rx90p[1]", "d2", "gauss", "amp")],
    [("rx90p[1]", "d2", "gauss", "freq_offset")],
    [("rx90p[1]", "d2", "gauss", "xy_angle")],
    [("rx90p[1]", "d2", "gauss", "delta")],
    [("rx90p[1]", "d2", "carrier", "framechange")],
]

parameter_map.set_opt_map(gateset_opt_map)

parameter_map.print_parameters()
exp.set_opt_gates(opt_gates)
opt.set_exp(exp)
opt.optimize_controls()
parameter_map.print_parameters()
print(opt.current_best_goal)

rx90p[0]-d1-gauss-amp                 : 500.000 mV 
rx90p[0]-d1-gauss-freq_offset         : -53.000 MHz 2pi 
rx90p[0]-d1-gauss-xy_angle            : -444.089 arad 
rx90p[0]-d1-gauss-delta               : -1.000  
rx90p[0]-d1-carrier-framechange       : 0.000 rad 
rx90p[1]-d2-gauss-amp                 : 500.000 mV 
rx90p[1]-d2-gauss-freq_offset         : -53.000 MHz 2pi 
rx90p[1]-d2-gauss-xy_angle            : -444.089 arad 
rx90p[1]-d2-gauss-delta               : -1.000  
rx90p[1]-d2-carrier-framechange       : 0.000 rad 

C3:STATUS:Saving as: /tmp/tmp95zhm6yh/c3logs/cnot12/2022_08_17_T_13_22_54/open_loop.c3log
rx90p[0]-d1-gauss-amp                 : 385.864 mV 
rx90p[0]-d1-gauss-freq_offset         : -52.985 MHz 2pi 
rx90p[0]-d1-gauss-xy_angle            : 148.166 mrad 
rx90p[0]-d1-gauss-delta               : -963.792 m 
rx90p[0]-d1-carrier-framechange       : -293.506 mrad 
rx90p[1]-d2-gauss-amp                 : 378.861 mV 
rx90p[1]-d2-gauss-freq_offset         : -52.984 MHz 2pi 
rx

## Calibration

In [8]:
from c3.qiskit.c3_gates import RX90pGate, RX90mGate, RY90pGate, RY90mGate, SetParamsGate
from qiskit import QuantumCircuit
from typing import List
QISKIT_GATE_MAP = { "rx90p": RX90pGate, "rx90m": RX90mGate, "ry90p": RY90pGate, "ry90m": RY90mGate}
def seqs_to_circuit(seqs: List[List[str]]) -> QuantumCircuit:
    circuits = []
    for seq in seqs:
        qc_sec = QuantumCircuit(1, 1)
        for gate in seq:
            qc_sec.append(QISKIT_GATE_MAP[gate[:-3]](), [int(gate[-2])])
        circuits.append(qc_sec)
    return circuits

In [10]:
import numpy as np
import tensorflow as tf
from c3.qiskit import C3Provider
from c3.utils import qt_utils

### ORBIT meta-parameters ###
RB_length = 12 # How long each sequence is
RB_number = 5  # How many sequences
shots = 1000    # How many averages per readout

orbit_provider = C3Provider()
orbit_exp = Experiment()
orbit_exp.read_config("two_qubits_TLS.c3exp")

gates = []
rx90p_q1 = orbit_exp.pmap.instructions["rx90p[0]"]
gates.extend(generate_gateset(rx90p_q1))
rx90p_q2 = orbit_exp.pmap.instructions["rx90p[1]"]
gates.extend(generate_gateset(rx90p_q2))
for gate in gates:
    orbit_exp.pmap.instructions[gate.get_key()] = gate
orbit_exp.pmap.update_parameters()

orbit_backend = orbit_provider.get_backend("c3_qasm_physics_simulator")
orbit_backend.set_c3_experiment(orbit_exp)

def ORBIT_qiskit(params):
    
    populations = []
    results = []
    results_std = []
    shots_nums = []

    # Creating the RB sequences
    seqs = qt_utils.single_length_RB(
            RB_number=RB_number, RB_length=RB_length, target=0
    )
    orbit_exp.set_opt_gates_seq(seqs) # speeds up the simulation of circuits
    circuits = seqs_to_circuit(seqs)

    orbit_job = orbit_backend.run(circuits, params = params, opt_map = gateset_opt_map)
    populations = [list(result.data.state_pops.values()) for result in orbit_job.result().results]
        
    for pop in populations:
        excited_pop = np.array(pop[1:]).sum() # total excited states population
        results.append(np.array([excited_pop]))
        results_std.append([0])
        shots_nums.append([shots])

    goal = np.mean(results) # average of the excited state populations from every circuit
    return goal, results, results_std, seqs, shots_nums

/home/ubuntu/c3/c3/signal/gates.py:135: UserWarning: No ideal gate found for gate:  . Use set_ideal() explicitly.
  warnings.warn(


In [11]:
from c3.libraries.algorithms import cmaes
from c3.optimizers.calibration import Calibration
alg_options = {
    "popsize" : 10,
    "maxfevals" : 300,
    "init_point" : "True",
    "tolfun" : 0.01,
    "spread" : 0.1
}
state_labels = {
      "excited" : [(1,), (2,)]
  }
# Create a temporary directory to store logfiles, modify as needed
log_dir = "Entangling_TLS"

opt = Calibration(
    dir_path=log_dir,
    run_name="ORBIT_cal",
    eval_func=ORBIT_qiskit,
    pmap=parameter_map,
    exp_right=exp,
    algorithm=cmaes,
    options=alg_options
)
opt.optimize_controls()

C3:STATUS:Saving as: /home/ubuntu/c3/examples/Entangling_TLS/ORBIT_cal/2022_08_17_T_13_26_11/calibration.log
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=825239, Wed Aug 17 13:26:11 2022)
C3:STATUS:Adding initial point to CMA sample.
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     11 5.726897242255344e-01 1.0e+00 9.24e-02  9e-02  1e-01 0:36.6
    2     21 5.344317031464187e-01 1.2e+00 9.05e-02  9e-02  1e-01 1:11.6
    3     31 5.061912229056994e-01 1.2e+00 9.12e-02  9e-02  1e-01 1:46.0
    4     41 5.919121947885373e-01 1.4e+00 9.75e-02  9e-02  1e-01 2:20.1
    5     51 5.332081141020383e-01 1.5e+00 9.87e-02  9e-02  1e-01 2:56.4
    6     61 5.903629320836812e-01 1.6e+00 9.81e-02  9e-02  1e-01 3:31.9
    7     71 5.644743548200941e-01 1.6e+00 9.46e-02  8e-02  1e-01 4:07.6
    8     81 6.404194730151364e-01 1.6e+00 9.69e-02  8e-02  1e-01 4:44.0
    9     91 5.146324346372372e-01 1.7e+00 9.92e-02  8e-02  1e-01 5:19.1
   10    101 5.909134868

In [12]:
exp_opt_map = [
    [('Q1', 'anhar')], 
    [('Q1', 'freq')],
    [('Q2', 'anhar')], 
    [('Q2', 'freq')],
    [('Q1-Q2', 'strength')], 
]
parameter_map.set_opt_map(exp_opt_map)

In [13]:
parameter_map.print_parameters()

Q1-anhar                              : -210.000 MHz 2pi 
Q1-freq                               : 5.000 GHz 2pi 
Q2-anhar                              : -240.000 MHz 2pi 
Q2-freq                               : 5.600 GHz 2pi 
Q1-Q2-strength                        : 50.000 MHz 2pi 



In [20]:
DATAFILE_PATH = "Entangling_TLS/recent/dataset.pickle"
datafiles = {"orbit": DATAFILE_PATH} # path to the dataset
run_name = "simple_model_learning" # name of the optimization run
dir_path = "ml_logs" # path to save the learning logs
algorithm = "cma_pre_lbfgs" # algorithm for learning
# this first does a grad-free CMA-ES and then a gradient based LBFGS
options = {
    "cmaes": {
        "popsize": 12,
        "init_point": "True",
        "stop_at_convergence": 10,
        "ftarget": 4,
        "spread": 0.05,
        "stop_at_sigma": 0.01,
    },
    "lbfgs": {"maxfun": 50, "disp": 0},
} # options for the algorithms
sampling = "high_std" # how data points are chosen from the total dataset
batch_sizes = {"orbit": 2} # how many data points are chosen for learning
state_labels = {
    "orbit": [
        (1,0),
        (2,0),
        (1,1),
        (2,1),
        (1,2),
        (2,2),
    ]
} # the excited states of the qubit model, in this case it is 3-level

In [18]:
from c3.optimizers.modellearning import ModelLearning

In [ ]:
opt = ModelLearning(
    datafiles=datafiles,
    run_name=run_name,
    dir_path=dir_path,
    algorithm=algorithm,
    options=options,
    sampling=sampling,
    batch_sizes=batch_sizes,
    state_labels=state_labels,
    pmap=parameter_map,
)

opt.set_exp(exp)

In [19]:
opt.run()

C3:STATUS:Saving as: /home/ubuntu/c3/examples/ml_logs/simple_model_learning/2022_08_17_T_13_50_22/model_learn.log
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 5 (seed=822229, Wed Aug 17 13:50:22 2022)
C3:STATUS:Adding initial point to CMA sample.
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     13 7.263387700877038e+02 1.0e+00 5.21e-02  5e-02  6e-02 0:08.2
    2     25 6.162547033365493e+02 1.5e+00 6.26e-02  6e-02  8e-02 0:15.8
    3     37 5.726184101929803e+02 2.0e+00 7.73e-02  7e-02  1e-01 0:23.4
    4     49 4.286613406523711e+02 2.7e+00 1.06e-01  9e-02  2e-01 0:30.7
    5     61 3.857329890844186e+02 2.8e+00 1.36e-01  1e-01  2e-01 0:38.3
    6     73 4.048276886919442e+02 2.8e+00 1.51e-01  1e-01  2e-01 0:45.8
    7     85 4.020732531748209e+02 2.4e+00 1.64e-01  1e-01  2e-01 0:53.2
    8     97 3.639956805575007e+02 2.4e+00 1.72e-01  1e-01  2e-01 1:00.7
   10    121 2.795808872879667e+02 2.5e+00 1.87e-01  1e-01  2e-01 1:15.7
   12    145 1.53098

In [10]:
from c3.libraries import chip, hamiltonians
from c3.c3objs import Quantity

TLS = chip.Qubit(
    name="TLS",
    desc="Spectator",
    freq=Quantity(
        value=4.5e9,
        min_val=4e9,
        max_val=6e9,
        unit="Hz 2pi",
    ),
    anhar=Quantity(
        value=-250e6,
        min_val=-380e6,
        max_val=-120e6,
        unit="Hz 2pi",
    ),
    hilbert_dim=3,
    temp=Quantity(value=0.05, min_val=0.0, max_val=0.12, unit="K"),
)

TLS_Q1 = chip.Coupling(
    name="TLS-Q1",
    strength=Quantity(
        value=5e6,
        min_val=1e6,
        max_val=150e6,
        unit="Hz 2pi",
    ),
    hamiltonian_func=hamiltonians.int_XX,
    connected=["Q1", "TLS"]
)


In [11]:
model = exp.pmap.model

In [12]:
subs = list(model.subsystems.values())
subs.append(TLS)
coups = list(model.couplings.values())
coups.append(TLS_Q1)
model.set_components(subsystems=subs, couplings=coups)

In [13]:
model.update_model()

In [14]:
parameter_map.update_parameters()

In [15]:
exp_opt_map = [
    [('Q1', 'anhar')], 
    [('Q1', 'freq')],
    [('Q2', 'anhar')], 
    [('Q2', 'freq')],
    [('Q1-Q2', 'strength')],
    [('TLS', 'anhar')], 
    [('TLS', 'freq')],
    [('TLS-Q1', 'strength')],
]
parameter_map.set_opt_map(exp_opt_map)

In [16]:
parameter_map.print_parameters()

Q1-anhar                              : -210.000 MHz 2pi 
Q1-freq                               : 5.000 GHz 2pi 
Q2-anhar                              : -240.000 MHz 2pi 
Q2-freq                               : 5.600 GHz 2pi 
Q1-Q2-strength                        : 50.000 MHz 2pi 
TLS-anhar                             : -250.000 MHz 2pi 
TLS-freq                              : 4.500 GHz 2pi 
TLS-Q1-strength                       : 5.000 MHz 2pi 



In [23]:
model.subsystems

{'Q1': <c3.libraries.chip.Qubit at 0x7f621c154e20>,
 'Q2': <c3.libraries.chip.Qubit at 0x7f621c154a00>,
 'TLS': <c3.libraries.chip.Qubit at 0x7f621c0460a0>}

In [24]:
state_labels = {
    "orbit": [
        (1,0,0),
        (2,0,0),
        (1,1,0),
        (2,1,0),
        (1,2,0),
        (2,2,0),
    ]
}

opt = ModelLearning(
    datafiles=datafiles,
    run_name=run_name,
    dir_path=dir_path,
    algorithm=algorithm,
    options=options,
    sampling=sampling,
    batch_sizes=batch_sizes,
    state_labels=state_labels,
    pmap=parameter_map,
)

opt.set_exp(exp)
opt.run()

C3:STATUS:Saving as: /home/ubuntu/c3/examples/ml_logs/simple_model_learning/2022_08_18_T_13_38_50/model_learn.log
(6_w,12)-aCMA-ES (mu_w=3.7,w_1=40%) in dimension 8 (seed=959639, Thu Aug 18 13:38:50 2022)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     12 4.654139073977564e+02 1.0e+00 1.05e-01  1e-01  1e-01 0:34.5
    2     24 4.568581024097264e+02 1.3e+00 1.10e-01  1e-01  1e-01 1:09.0
    3     36 4.032524531225563e+02 1.4e+00 1.25e-01  1e-01  2e-01 1:43.9
    4     48 3.556278134963339e+02 1.7e+00 1.31e-01  1e-01  2e-01 2:18.3
    5     60 2.816695389915497e+02 1.8e+00 1.61e-01  2e-01  2e-01 2:53.3
    6     72 2.492270516828018e+02 1.8e+00 1.95e-01  2e-01  2e-01 3:28.4
    7     84 2.589367014418652e+02 1.8e+00 2.53e-01  2e-01  3e-01 4:03.0
    8     96 1.602653598219491e+02 1.9e+00 3.06e-01  3e-01  4e-01 4:37.2
    9    108 1.697284890486517e+02 1.8e+00 3.07e-01  3e-01  4e-01 5:11.1
   10    120 1.732584901254043e+02 2.0e+00 3.34e-01  3e-01  4e-01 

/home/ubuntu/c3/c3/model.py:433: UserWarning: C3 Warning: Some states are overly dressed, trying to recover...7 states, 4 is lowest failed state
  warnings.warn(


   17    204 1.629575928942047e+02 2.4e+00 3.14e-01  2e-01  4e-01 9:45.9
   18    216 2.471155847961675e+02 2.4e+00 2.94e-01  2e-01  3e-01 10:20.2
   19    228 2.014289767774075e+02 2.4e+00 2.72e-01  2e-01  3e-01 10:54.9
   20    240 1.781835783448459e+02 2.4e+00 2.26e-01  1e-01  2e-01 11:29.2
   21    252 1.788104500065221e+02 2.5e+00 2.18e-01  1e-01  2e-01 12:03.4
   22    264 1.856255152954863e+02 2.5e+00 2.02e-01  1e-01  2e-01 12:38.0
   23    276 1.625344922882413e+02 2.7e+00 2.15e-01  1e-01  2e-01 13:13.6
   24    288 1.587819894215189e+02 2.7e+00 2.04e-01  1e-01  2e-01 13:49.5
   25    300 1.649021475758377e+02 2.6e+00 1.90e-01  1e-01  2e-01 14:25.5
   26    312 1.567271577172041e+02 2.5e+00 1.72e-01  9e-02  2e-01 15:00.7
   27    324 1.559483834461470e+02 2.6e+00 1.63e-01  9e-02  2e-01 15:35.5
   28    336 1.553247577646513e+02 2.7e+00 1.73e-01  9e-02  2e-01 16:10.5
   29    348 1.579352941381832e+02 3.1e+00 1.56e-01  8e-02  2e-01 16:45.0
   30    360 1.736578090584323e+02 3.0e

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /home/ubuntu/c3/c3/parametermap.py:310 in set_parameters                                  │
│                                                                                           │
│   307 │   │   │   │   except ValueError as ve:                                            │
│   308 │   │   │   │   │   raise Exception(f"C3:ERROR:{key} not defined.") from ve         │
│   309 │   │   │   │   try:                                                                │
│ ❱ 310 │   │   │   │   │   par.set_value(values[val_indx], extend_bounds=extend_bounds)    │
│   311 │   │   │   │   except (ValueError, InvalidArgumentError) as ve:                    │
│   312 │   │   │   │   │   try:                                                            │
│   313 │   │   │   │   │   │   raise Exception(                                            │
│ /home/ubuntu/c3/c3/c3objs.py:269 in set_value                                             │
│                                                                                           │
│   266 │   │   if extend_bounds:                                                           │
│   267 │   │   │   self._set_value_extend(np.reshape(val, self.shape))                     │
│   268 │   │   else:                                                                       │
│ ❱ 269 │   │   │   self._set_value(np.reshape(val, self.shape))                            │
│   270 │                                                                                   │
│   271 │   def _set_value(self, val) -> None:                                              │
│   272 │   │   """Set the value of this quantity as tensorflow. Value needs to be          │
│                                                                                           │
│ /home/ubuntu/c3/c3/c3objs.py:285 in _set_value                                            │
│                                                                                           │
│   282 │   │   )                                                                           │
│   283 │   │                                                                               │
│   284 │   │   if np.any(tf.math.abs(tmp) > tf.constant(1.0, tf.float64)):                 │
│ ❱ 285 │   │   │   raise ValueError(                                                       │
│   286 │   │   │   │   f"Value {num3str(val.numpy())}{self.unit} out of bounds for quantit │
│   287 │   │   │   │   f"min_val: {num3str(self.get_limits()[0])}{self.unit} and "         │
│   288 │   │   │   │   f"max_val: {num3str(self.get_limits()[1])}{self.unit}",             │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Value -51.900 MHz 2pi out of bounds for quantity with min_val: -56.000 MHz 2pi 
and max_val: -52.000 MHz 2pi

The above exception was the direct cause of the following exception:

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_3290347/2745720778.py:25 in <cell line: 25>                                │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_3290347/2745720778.py'               │
│ /home/ubuntu/c3/c3/optimizers/modellearning.py:200 in learn_model                         │
│                                                                                           │
│   197 │   │   except KeyboardInterrupt:                                                   │
│   198 │   │   │   pass                                                                    │
│   199 │   │   self.end_log()                  